In [2]:
# Standard libraries
import h5py
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

# Append base directory
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
path1p = os.path.dirname(currentdir)
path2p = os.path.dirname(path1p)
libpath = os.path.join(path2p, "lib")

sys.path.insert(0, libpath)
print("Appended library directory", libpath)

# User libraries
from data_io.os_lib import getfiles_walk
from data_io.yaro.yaro_data_read import read_neuro_perf
from signal_lib import resample
from fc.te_idtxl_wrapper import idtxlParallelCPUMulti, idtxlResultsParse
from qt_wrapper import gui_fpath, gui_fnames

%load_ext autoreload
%autoreload 2

Appended library directory /home/alyosha/work/git/mesoscopic-functional-connectivity/codes/lib


## 1. Load Neuronal Data

The neuronal data for a given experiment is given by the 3D matrix of dimensions `[N_Trial, N_Times, N_Channels]`

Notes:
* `DT = 0.05` : the sampling interval is always 50ms
* `N_Times = 201` : the total experiment time is always 10s. All experiments are aligned

In [36]:
idtxl_settings = {
    'dim_order'       : 'rsp',
    'methods'         : ["BivariateTE", "MultivariateTE"],
    'cmi_estimator'   : 'JidtGaussianCMI',
    'min_lag_sources' : 1,
    'max_lag_sources' : 5
}

In [35]:
# Get paths
datafilenames = gui_fnames("Simulated Data files", "./", "HDF5 files(*.h5)")

In [5]:
out_path  = gui_fpath("Path where to save results", "./")

In [37]:
for fname in datafilenames:
    # Read data
    h5f = h5py.File(fname, "r")
    data = np.copy(h5f['data'])
    h5f.close()
    
    # Get parameters
    nTrials, nTimes, nChannels = data.shape
    print("Loaded neuronal data with (nTrials, nTimes, nChannels)=", data.shape)
    
    #############################
    # Analysis
    #############################

    teWindow = idtxl_settings["max_lag_sources"] + 1

    data_range = list(range(nTimes - teWindow + 1))
    data_lst = [data[:, i:i + teWindow, :] for i in data_range]
    rez = idtxlParallelCPUMulti(data_lst, idtxl_settings, os.path.basename(fname)[:-4])

    for iMethod, method in enumerate(idtxl_settings['methods']):
        te_data = np.full((3, nChannels, nChannels, nTimes), np.nan)

        for iRange in data_range:
            te_data[..., iRange + idtxl_settings["max_lag_sources"]] = np.array(
                idtxlResultsParse(rez[iMethod][iRange], nChannels, method=method, storage='matrix')
            )

        #######################
        # Save results to file
        #######################
        savename = os.path.join(out_path, 'swipe_' + method + '_' + os.path.basename(fname[:-3]) + '.h5')
        print(savename)

        h5f = h5py.File(savename, "w")

        grp_rez = h5f.create_group("results")
        grp_rez['TE_table']    = te_data[0]
        grp_rez['delay_table'] = te_data[1]
        grp_rez['p_table']     = te_data[2]

        h5f.close()

Loaded neuronal data with (nTrials, nTimes, nChannels)= (400, 50, 12)
Adding data with properties: 12 processes, 6 samples, 400 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 400 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 400 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 400 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 400 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 400 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 400 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 400 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 400 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 400